In [84]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv -O leads_coring.csv


--2025-10-14 01:57:29--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: 'leads_coring.csv'

     0K .......... .......... .......... .......... .......... 63%  619K 0s
    50K .......... .......... ........                        100% 4,08M=0,09s

2025-10-14 01:57:30 (900 KB/s) - 'leads_coring.csv' saved [80876/80876]



In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [86]:
df = pd.read_csv('leads_coring.csv')
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [87]:
df.converted.isnull().sum()

np.int64(0)

In [88]:
len(df)-df.notnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [89]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [90]:
df.dtypes[df.dtypes == 'object']

lead_source          object
industry             object
employment_status    object
location             object
dtype: object

In [91]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [92]:
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NaN,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NaN,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [93]:
numeric_columns = list(df.dtypes[df.dtypes != 'object'].index)
numeric_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [94]:
df[categorical_columns].isna().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [95]:
df[categorical_columns]= df[categorical_columns].fillna('NA')
df[categorical_columns].isna().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [96]:
df[numeric_columns].isnull().sum()

number_of_courses_viewed      0
annual_income               181
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [97]:
df[numeric_columns] = df[numeric_columns].fillna(0)
df[numeric_columns].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [98]:
df['industry'].nunique()

8

## Question 1
What is the most frequent observation (mode) for the column industry?

In [99]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

Split the data
Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value converted is not in your dataframe.

In [100]:
from sklearn.model_selection import train_test_split

In [101]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [102]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [103]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [104]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [117]:
numeric_columns.pop()
numeric_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

## Question 2.
 Biggest correlation (1 point)

In [156]:
df_full_train[numeric_columns].corrwith(df_full_train.converted).abs()

number_of_courses_viewed    0.442068
annual_income               0.029612
interaction_count           0.378482
lead_score                  0.225641
dtype: float64

## Question 3
Calculate the mutual information score between converted and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).

In [105]:
from sklearn.metrics import mutual_info_score

In [106]:
df_full_train

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
1066,social_media,manufacturing,2,44403.0,self_employed,australia,1,0.71,0
638,events,retail,3,38048.0,student,north_america,6,0.97,1
799,social_media,education,2,71399.0,NA,europe,1,0.51,1
380,referral,education,2,47912.0,employed,australia,1,0.04,0
303,paid_ads,healthcare,1,34806.0,employed,europe,4,0.32,1
...,...,...,...,...,...,...,...,...,...
1130,events,manufacturing,1,57039.0,employed,south_america,2,0.30,0
1294,events,healthcare,2,56185.0,student,south_america,2,0.44,0
860,paid_ads,manufacturing,1,56402.0,student,north_america,1,0.02,0
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1


In [109]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_full_train.converted)

mi = df_full_train[categorical_columns].apply(mutual_info_converted_score)
round(mi.sort_values(ascending=False),2)

lead_source          0.03
employment_status    0.01
industry             0.01
location             0.00
dtype: float64

## Question 4
Now let's train a logistic regression.
Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
Fit the model on the training dataset.
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [112]:
from sklearn.feature_extraction import DictVectorizer

In [116]:
numeric_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [118]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numeric_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numeric_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [122]:
from sklearn.linear_model import LogisticRegression

In [124]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [125]:
model.intercept_[0]

np.float64(-0.06914728027832363)

In [126]:
model.coef_[0].round(2)

array([-0.  , -0.01,  0.03,  0.  ,  0.01, -0.1 , -0.02,  0.05, -0.02,
       -0.01, -0.  , -0.01, -0.03, -0.02,  0.31,  0.05,  0.02, -0.01,
       -0.01, -0.12,  0.08, -0.03,  0.  , -0.01, -0.01, -0.01,  0.01,
        0.01, -0.03, -0.03,  0.45])

In [139]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.61192163, 0.79982617, 0.53021344, 0.47131479, 0.57066131,
       0.44227168, 0.87127669, 0.84883115, 0.83290037, 0.61497801,
       0.54968026, 0.78153088, 0.69039785, 0.77017122, 0.5265944 ,
       0.91706425, 0.53170634, 0.42123048, 0.30146455, 0.84881583,
       0.79488653, 0.73670374, 0.44527211, 0.64838383, 0.4176882 ,
       0.75393418, 0.90166116, 0.33903048, 0.43181431, 0.9680681 ,
       0.92018714, 0.37487988, 0.652301  , 0.90650057, 0.75164116,
       0.64202121, 0.82250075, 0.83375553, 0.659116  , 0.30978853,
       0.78942264, 0.35546366, 0.96517758, 0.63389304, 0.51274195,
       0.53230533, 0.82287785, 0.744074  , 0.73452313, 0.68955217,
       0.46964443, 0.84539252, 0.55635243, 0.92637871, 0.65258021,
       0.61526273, 0.63816995, 0.28304018, 0.48049824, 0.57890618,
       0.35497342, 0.62175051, 0.38960778, 0.61156056, 0.85304278,
       0.75430136, 0.89185954, 0.71946459, 0.95387623, 0.89209517,
       0.75277087, 0.33850139, 0.61376593, 0.51622275, 0.64088

In [137]:
y_pred = model.predict(X_val)
CONVERTED_decision = (y_pred >= 1)

In [140]:
CONVERTED_decision = (y_pred >= 0.5)

In [142]:
# accuracy of validation set
round((y_val == CONVERTED_decision).mean(),4)

np.float64(0.6997)

## Question 5
 - Let's find the least useful feature using the feature elimination technique.
 - Train a model using the same features and parameters as in Q4 (without rounding).
 - Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
 - For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

 - 'industry'
 - 'employment_status'
 - 'lead_score'
  
Note: The difference doesn't have to be positive.

feature elimination technique:

In [161]:
features_dict = dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)) )
features_dict

{'annual_income': np.float64(-0.0),
 'employment_status=NA': np.float64(-0.015),
 'employment_status=employed': np.float64(0.034),
 'employment_status=self_employed': np.float64(0.003),
 'employment_status=student': np.float64(0.012),
 'employment_status=unemployed': np.float64(-0.103),
 'industry=NA': np.float64(-0.025),
 'industry=education': np.float64(0.049),
 'industry=finance': np.float64(-0.02),
 'industry=healthcare': np.float64(-0.013),
 'industry=manufacturing': np.float64(-0.003),
 'industry=other': np.float64(-0.009),
 'industry=retail': np.float64(-0.032),
 'industry=technology': np.float64(-0.016),
 'interaction_count': np.float64(0.311),
 'lead_score': np.float64(0.051),
 'lead_source=NA': np.float64(0.02),
 'lead_source=events': np.float64(-0.012),
 'lead_source=organic_search': np.float64(-0.012),
 'lead_source=paid_ads': np.float64(-0.115),
 'lead_source=referral': np.float64(0.08),
 'lead_source=social_media': np.float64(-0.03),
 'location=NA': np.float64(0.004),
 'l

## Question 5

 - Let's find the least useful feature using the feature elimination technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

- 'industry'
- 'employment_status'
- 'lead_score'
  
Note: The difference doesn't have to be positive.

## Question 6
 - Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

In [162]:
dicts_full_train = df_full_train[categorical_columns + numeric_columns].to_dict(orient='records')

In [163]:
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

In [164]:
y_full_train = df_full_train.converted.values

In [184]:
def reg_log_regression(C_parameter,X_full_train, y_full_train,df_test):
    model = LogisticRegression(solver='lbfgs',max_iter=10000, C=C_parameter)
    model.fit(X_full_train, y_full_train)
    dicts_test = df_test[categorical_columns + numeric_columns].to_dict(orient='records')
    X_test = dv.transform(dicts_test)
    y_pred = model.predict_proba(X_test)[:, 1]
    CONVERTED_decision = (y_pred >= 0.5)
    return (CONVERTED_decision == y_test).mean()

In [188]:
for c in [0.01, 0.1, 1, 10, 100]:
    accuracy  = reg_log_regression(c, X_full_train, y_full_train, df_test)
    print(f'c = {c}\t accuracy ={round(accuracy ,3)} ')

c = 0.01	 accuracy =0.819 
c = 0.1	 accuracy =0.829 
c = 1	 accuracy =0.823 
c = 10	 accuracy =0.816 
c = 100	 accuracy =0.816 
